(seq:pyalf_comprun)=
# Compiling and running ALF
This section focuses on the "ALF interface" part of pyALF, i.e. how to compile ALF and run ALF simulations. This revolves around the classes `ALF_source` and `Simulation` defined in the module `py_alf` that have already been briefly introduced in {numref}`sec:pyalf_minimal_example`.

We start with some imports:

In [1]:
from pprint import pprint  # Pretty print

from py_alf import ALF_source, Simulation  # Interface with ALF

## Class `ALF_source`

The Class {class}`py_alf.ALF_source` points to a folder containing the ALF source code. It has the following signature:

```Python
class ALF_source(
    alf_dir=os.getenv('ALF_DIR', './ALF'),
    branch=None,
    url='https://git.physik.uni-wuerzburg.de/ALF/ALF.git'
)
```

Where `os.getenv('ALF_DIR', './ALF')` gets the environment variable `$ALF_DIR` if present and otherwise returns `'./ALF'`. If the directory `alf_dir` does exist, the program assumes it contains
the ALF source code and will raise an Exception if that is not the case. If `alf_dir` does not exist, the
source code will be cloned form `url`. If `branch` is set, git checks it out.

We will just use the default:

In [2]:
alf_src = ALF_source()

And see if it successfully found ALF:

In [3]:
alf_src.alf_dir

'/home/jonas/Programs/ALF'

We can use the function {func}`py_alf.ALF_source.get_ham_names` to see which Hamiltonians are implemented:

In [4]:
alf_src.get_ham_names()

['Kondo',
 'Hubbard',
 'Hubbard_Plain_Vanilla',
 'tV',
 'LRC',
 'Z2_Matter',
 'Spin_Peierls']

And then view the list of parameters and their default values for a particular Hamiltonian. The Hamiltonian-specific parameters are listed first, followed by the Hamiltonian-independent parameters.

In [5]:
pprint(alf_src.get_default_params('Hubbard'))

OrderedDict([('VAR_lattice',
              {'L1': {'comment': 'Length in direction a_1',
                      'defined_in_base': False,
                      'value': 6},
               'L2': {'comment': 'Length in direction a_2',
                      'defined_in_base': False,
                      'value': 6},
               'Lattice_type': {'comment': '',
                                'defined_in_base': False,
                                'value': 'Square'},
               'Model': {'comment': 'Value not relevant',
                         'defined_in_base': False,
                         'value': 'Hubbard'}}),
             ('VAR_Model_Generic',
              {'Beta': {'comment': 'Inverse temperature',
                        'defined_in_base': False,
                        'value': 5.0},
               'Bulk': {'comment': 'Twist as a vector potential (.T.), or at '
                                   'the boundary (.F.)',
                        'defined_in_base': False,
   

## Class `Simulation`

To set up a simulation, we create an instance of {class}`py_alf.Simulation`, which has the signature

```Python
class Simulation(alf_src, ham_name, sim_dict, **kwargs)
```

where `alf_src` is an instance of {class}`py_alf.ALF_source`, `ham_name` is the name of the Hamiltonian to simulate, `sim_dict` is a dictionary of `parameter: value` pairs overwriting the default parameters and `**kwargs` represents optional keyword arguments.

The minimal set of required arguments does not overwrite any default parameters:

In [6]:
sim = Simulation(alf_src, 'Hubbard', {})

Before running the simulation, ALF needs to be compiled.

In [7]:
sim.compile()

Compiling ALF... 
Cleaning up Prog/
Cleaning up Libraries/
Cleaning up Analysis/


Compiling Libraries


entanglement_mod.F90:35:2:

   35 | #warning "You are compiling entanglement without MPI. No Renyi entropy results possible, all other observables still work!"
      |  1~~~~~~


ar: creating modules_90.a


ar: creating libqrref.a


Compiling Analysis


Compiling Program
Parsing Hamiltonian parameters
filenames: Hamiltonians/Hamiltonian_Kondo_smod.F90 Hamiltonians/Hamiltonian_Kondo_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_smod.F90 Hamiltonians/Hamiltonian_Hubbard_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90 Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_tV_smod.F90 Hamiltonians/Hamiltonian_tV_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_LRC_smod.F90 Hamiltonians/Hamiltonian_LRC_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90 Hamiltonians/Hamiltonian_Z2_Matter_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Spin_Peierls_smod.F90 Hamiltonians/Hamiltonian_Spin_Peierls_read_write_parameters.F90
Compiling program modules


Link program
Done.


Preparation of the simulation is done by executing the following command:

In [8]:
sim.run()

Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard" for Monte Carlo run.
Create new directory.
Run /home/jonas/Programs/ALF/Prog/ALF.out
 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


It is strongly advised to take a look at the info file `info` produced by ALF after a finished run, in particular the value of "Precision Green" and "Precision Phase". As a rule of thumb, the means should be of order $10^{-8}$ or smaller and the max should not be bigger than $10^{-3}$. If they're bigger, one should decrease the stabilization interval `Nwrap` (see parameter list `'VAR_QMC'` above). In our case, they're about right.

In [9]:
sim.print_info_file()

===== /home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard/info =====
 Model is      : Hubbard                                                         
 Lattice is    : Square                                                          
 # unit cells  :           36
 # of orbitals :            1
 Flux_1        :    0.0000000000000000     
 Flux_2        :    0.0000000000000000     
 Twist as phase factor in bulk
 HS  couples to z-component of spin
 Checkerboard  :  T
 Symm. decomp  :  T
 Finite temperture version
 Beta          :    5.0000000000000000     
 dtau,Ltrot_eff:   0.10000000000000001               50
 N_SUN         :            2
 N_FL          :            2
 t             :    1.0000000000000000     
 Ham_U         :    4.0000000000000000     
 Ham_chem      :    0.0000000000000000     
 No initial configuration, Seed_in      790789
 Sweeps                              :           20
 Bins                                :            5
 No CPU-time limitation 
 Measure

## Specifying parameters

Here is an example of a simulation with non-default parameters. We have changed the dimensions to 4 by 4 sites and increased the interaction $U$ to $4.0$ and the number of bins calculated to 20. Since we did not change the compile-time configuration (some of the `**kwargs` do), a recompilation is not required.

In [10]:
sim = Simulation(
    alf_src,
    'Hubbard',
    {
        # Model specific parameters
        'L1': 4,
        'L2': 4,
        'Ham_U': 4.0,
        # QMC parameters
        'Nbin': 20,
    },
)
sim.run()

Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard_L1=4_L2=4_U=4.0" for Monte Carlo run.
Create new directory.
Run /home/jonas/Programs/ALF/Prog/ALF.out
 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


Note that the new simulation has been placed in `ALF_data/Hubbard_L1=4_L2=4_U=4.0` relative to the current working directory. 
That is, simulations are placed in the folder `{sim_root}/{sim_dir}`, where `sim_root` defaults to `'ALF_data'` and `sim_dir` is generated out of the Hamiltonian name and the non-default model specific parameters. A behavior that can be overwritten through the `**kwargs`. Note that `Nbin` does not enter `sim_dir`, since it is a QMC parameter and not a Hamiltonian parameter.

The monitoring in the info file does not show any stabilization issues:

In [11]:
sim.print_info_file()

===== /home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard_L1=4_L2=4_U=4.0/info =====
 Model is      : Hubbard                                                         
 Lattice is    : Square                                                          
 # unit cells  :           16
 # of orbitals :            1
 Flux_1        :    0.0000000000000000     
 Flux_2        :    0.0000000000000000     
 Twist as phase factor in bulk
 HS  couples to z-component of spin
 Checkerboard  :  T
 Symm. decomp  :  T
 Finite temperture version
 Beta          :    5.0000000000000000     
 dtau,Ltrot_eff:   0.10000000000000001               50
 N_SUN         :            2
 N_FL          :            2
 t             :    1.0000000000000000     
 Ham_U         :    4.0000000000000000     
 Ham_chem      :    0.0000000000000000     
 No initial configuration, Seed_in      790789
 Sweeps                              :           20
 Bins                                :           20
 No CPU-time limi

## Series of MPI runs

Starting each run separately can be cumbersome, therefore we provide the following example, which creates a list of `Simulation` instances that can be run in a loop, performing a sweep in $U$. To increase the statistics of the results, MPI parallelization is employed. Since the default MPI executable `mpiexec` does not fit with the MPI libraries used during compilation on the test machine, we have changed it to `orterun`. The option `mpiexec_args=['--oversubscribe']` hands over the flag `--oversubscribe` to `orterun`, which allows it to run more MPI tasks than there are slots available, see the [Open MPI documentation](https://www.open-mpi.org/doc) for details. 

In [12]:
sims = [
    Simulation(
        alf_src,
        'Hubbard',
        {
            # Model specific parameters
            'L1': 4,
            'L2': 4,
            'Ham_U': U,
            # QMC parameters
            'Nbin': 20,
        },
        mpi=True,
        n_mpi=4,
        # mpiexec='orterun',
        # mpiexec_args=['--oversubscribe'],
    )
    for U in [1.0, 2.0, 3.0]]

In [13]:
sims

````{note}
The above employs Python's {ref}`list comprehensions <python:tut-listcomps>`, a convenient and readable way to create Python lists. Here is a simple example, employing list comprehension (and {ref}`f-strings <python:f-strings>`):
```python
>>> [f'x={x}' for x in [1, 2, 3]]
['x=1', 'x=2', 'x=3']
```
````

Since we are changing from non-MPI to MPI, ALF has to be recompiled:

```{warning}
pyALF does not check how ALF has been compiled previously, so the user has to take care of issuing recompilation if necessary.
```

In [14]:
sims[0].compile()

Compiling ALF... 
Cleaning up Prog/
Cleaning up Libraries/


Cleaning up Analysis/
Compiling Libraries


ar: creating modules_90.a


ar: creating libqrref.a


Compiling Analysis


Compiling Program
Parsing Hamiltonian parameters
filenames: Hamiltonians/Hamiltonian_Kondo_smod.F90 Hamiltonians/Hamiltonian_Kondo_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_smod.F90 Hamiltonians/Hamiltonian_Hubbard_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90 Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_tV_smod.F90 Hamiltonians/Hamiltonian_tV_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_LRC_smod.F90 Hamiltonians/Hamiltonian_LRC_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90 Hamiltonians/Hamiltonian_Z2_Matter_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Spin_Peierls_smod.F90 Hamiltonians/Hamiltonian_Spin_Peierls_read_write_parameters.F90
Compiling program modules


Link program


Done.


Loop over list of jobs:

In [15]:
for sim in sims:
    sim.run()

Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard_L1=4_L2=4_U=1.0" for Monte Carlo run.
Create new directory.
Run /home/jonas/Programs/ALF/Prog/ALF.out


 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard_L1=4_L2=4_U=2.0" for Monte Carlo run.
Create new directory.
Run /home/jonas/Programs/ALF/Prog/ALF.out


 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard_L1=4_L2=4_U=3.0" for Monte Carlo run.
Create new directory.
Run /home/jonas/Programs/ALF/Prog/ALF.out


 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


In [16]:
for sim in sims:
    sim.print_info_file()

===== /home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard_L1=4_L2=4_U=1.0/info =====
 Model is      : Hubbard                                                         
 Lattice is    : Square                                                          
 # unit cells  :           16
 # of orbitals :            1
 Flux_1        :    0.0000000000000000     
 Flux_2        :    0.0000000000000000     
 Twist as phase factor in bulk
 HS  couples to z-component of spin
 Checkerboard  :  T
 Symm. decomp  :  T
 Finite temperture version
 Beta          :    5.0000000000000000     
 dtau,Ltrot_eff:   0.10000000000000001               50
 N_SUN         :            2
 N_FL          :            2
 t             :    1.0000000000000000     
 Ham_U         :    1.0000000000000000     
 Ham_chem      :    0.0000000000000000     
 No initial configuration, Seed_in      814342
 Sweeps                              :           20
 Bins                                :           20
 No CPU-time limi

## Parallel Tempering

ALF offers the possibility to employ Parallel Tempering&nbsp;{cite}`Greyer91`, also known as Exchange Monte Carlo&nbsp;{cite}`Hukushima96`, where simulations with different parameters but the same configuration space are run in parallel and can exchange configurations. A method developed to overcome ergodicity issues.

To use Parallel Tempering in pyALF, `sim_dict` has to be replaced by a list of dictionaries, for this we use again Python's list comprehension syntax. This does also imply `mpi=True`, since Parallel Tempering needs MPI.

In [17]:
sim = Simulation(
    alf_src,
    'Hubbard',
    [
        {
            # Model specific parameters
            'L1': 4,
            'L2': 4,
            'Ham_U': U,
            # QMC parameters
            'Nbin': 20,
            'mpi_per_parameter_set': 2
        } for U in [2.5, 3.5]
    ],
    mpi=True,
    n_mpi=4,
    # mpiexec='orterun',
    # mpiexec_args=['--oversubscribe'],
)

Recompile for Parallel Tempering:

In [18]:
sim.compile()

Compiling ALF... 
Cleaning up Prog/
Cleaning up Libraries/


Cleaning up Analysis/
Compiling Libraries


ar: creating modules_90.a


ar: creating libqrref.a


Compiling Analysis


Compiling Program
Parsing Hamiltonian parameters
filenames: Hamiltonians/Hamiltonian_Kondo_smod.F90 Hamiltonians/Hamiltonian_Kondo_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_smod.F90 Hamiltonians/Hamiltonian_Hubbard_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90 Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_tV_smod.F90 Hamiltonians/Hamiltonian_tV_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_LRC_smod.F90 Hamiltonians/Hamiltonian_LRC_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90 Hamiltonians/Hamiltonian_Z2_Matter_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Spin_Peierls_smod.F90 Hamiltonians/Hamiltonian_Spin_Peierls_read_write_parameters.F90
Compiling program modules


Link program


Done.


In [19]:
sim.run()

Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/temper_Hubbard_L1=4_L2=4_U=2.5" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/temper_Hubbard_L1=4_L2=4_U=2.5/Temp_0" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/temper_Hubbard_L1=4_L2=4_U=2.5/Temp_1" for Monte Carlo run.
Create new directory.
Run /home/jonas/Programs/ALF/Prog/ALF.out


 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


In [20]:
sim.print_info_file()

===== /home/jonas/Programs/pyALF/doc/source/usage/ALF_data/temper_Hubbard_L1=4_L2=4_U=2.5/Temp_0/info =====
 Model is      : Hubbard                                                         
 Lattice is    : Square                                                          
 # unit cells  :           16
 # of orbitals :            1
 Flux_1        :    0.0000000000000000     
 Flux_2        :    0.0000000000000000     
 Twist as phase factor in bulk
 HS  couples to z-component of spin
 Checkerboard  :  T
 Symm. decomp  :  T
 Finite temperture version
 Beta          :    5.0000000000000000     
 dtau,Ltrot_eff:   0.10000000000000001               50
 N_SUN         :            2
 N_FL          :            2
 t             :    1.0000000000000000     
 Ham_U         :    2.5000000000000000     
 Ham_chem      :    0.0000000000000000     
 No initial configuration, Seed_in      814342
 Sweeps                              :           20
 Bins                                :           20
 No

The output from this command has been omitted for brevity.

## Only preparing runs

In many cases, it might not be feasible to execute ALF directly through pyALF, for example when using an HPC scheduler, but one might still like to use pyALF for preparing the simulation directories. In this case the two options `copy_bin` and `only_prep` of {func}`py_alf.Simulation.run` come in handy. Here we also demonstrate the keyword arguments `sim_root` and `sim_dir`.

In [21]:
import numpy as np

JK_list = np.linspace(0.0, 3.0, num=11)
print(JK_list)

sims = [
    Simulation(
        alf_src,
        'Kondo',
        {
            "Model": "Kondo",
            "Lattice_type": "Bilayer_square",
            "L1": 16,
            "L2": 16,
            "Ham_JK": JK,
            "Ham_Uf": 1.,
            "Beta": 20.0,
            "Nsweep": 500,
            "NBin": 400,
            "Ltau": 0,
            "CPU_MAX": 48
        },
        mpi=True,
        sim_root="KondoBilayerSquareL16",
        sim_dir=f"JK{JK:2.1f}",
    ) for JK in JK_list
]

[0.  0.3 0.6 0.9 1.2 1.5 1.8 2.1 2.4 2.7 3. ]


Do not forget to recompile when switching from Parallel Tempering back to normal MPI runs.

In [22]:
sims[0].compile()

Compiling ALF... 
Cleaning up Prog/


Cleaning up Libraries/
Cleaning up Analysis/
Compiling Libraries


ar: creating modules_90.a


ar: creating libqrref.a


Compiling Analysis


Compiling Program
Parsing Hamiltonian parameters
filenames: Hamiltonians/Hamiltonian_Kondo_smod.F90 Hamiltonians/Hamiltonian_Kondo_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_smod.F90 Hamiltonians/Hamiltonian_Hubbard_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90 Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_tV_smod.F90 Hamiltonians/Hamiltonian_tV_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_LRC_smod.F90 Hamiltonians/Hamiltonian_LRC_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90 Hamiltonians/Hamiltonian_Z2_Matter_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Spin_Peierls_smod.F90 Hamiltonians/Hamiltonian_Spin_Peierls_read_write_parameters.F90
Compiling program modules


Link program


Done.


In [23]:
for sim in sims:
    sim.run(copy_bin=True, only_prep=True)

Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/KondoBilayerSquareL16/JK0.0" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/KondoBilayerSquareL16/JK0.3" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/KondoBilayerSquareL16/JK0.6" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/KondoBilayerSquareL16/JK0.9" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/KondoBilayerSquareL16/JK1.2" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/KondoBilayerSquareL16/JK1.5" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/KondoBilayerSquareL16/JK1.8" for Monte Carlo run.
Create new directory.
Prepare directory "/home/jonas/Programs/pyALF/doc/sourc

Now there are 11 directories, ready for the job scheduler.

In [24]:
!tree KondoBilayerSquareL16

KondoBilayerSquareL16
├── JK0.0
│   ├── ALF.out
│   ├── parameters
│   └── seeds
├── JK0.3
│   ├── ALF.out
│   ├── parameters
│   └── seeds
├── JK0.6
│   ├── ALF.out
│   ├── parameters
│   └── seeds
├── JK0.9
│   ├── ALF.out
│   ├── parameters
│   └── seeds
├── JK1.2
│   ├── ALF.out
│   ├── parameters
│   └── seeds
├── JK1.5
│   ├── ALF.out
│   ├── parameters
│   └── seeds
├── JK1.8
│   ├── ALF.out
│   ├── parameters
│   └── seeds
├── JK2.1
│   ├── ALF.out
│   ├── parameters
│   └── seeds
├── JK2.4
│   ├── ALF.out
│   ├── parameters
│   └── seeds
├── JK2.7
│   ├── ALF.out
│   ├── parameters
│   └── seeds
└── JK3.0
    ├── ALF.out
    ├── parameters
    └── seeds

12 directories, 33 files
